## Document/Text processing and embedding creation

Ingredients:
* PDF document of choice
* Embedding model of choice

Steps :
1. Import PDF document
2. Process text for embedding (split into chunks of sentences)
3. Embed text chunks with embedding model
4. Save embeddings to file for late (embedding will store on file for many years or until you lose your hard drive)


In [ ]:
# # Import a PDF document
# import os
# import requests

# # Get the pdf document path

# pdf_path = "human-nutrition-text.pdf"

# #Download
# if not os.path.exists(pdf_path):
#   print(f"[INFO] File doesnt exist, downloading...")

#   #Enter the URL for download
#   url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

#   #the local filename to save the downloaded file
#   filename = pdf_path

#   #send a get request
#   response = requests.get(url)

#   #check if request was successful
#   if response.status_code == 200:
#     #open the file and save it

#     with open(filename, "wb") as file:
#       file.write(response.content)

#     print(f"[INFO] The file has been downloaded and saved as {filename}")
#   else:
#     print(f"[INFO] Failed to download the file. Status code {response.status_code}")

# else:
#   print(f"File already exists")

In [ ]:
!pip install pymupdf sentence_transformers transformers

Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 11.0 MB/s eta 0:00:00


In [ ]:
# !pip install pymupdf sentence_transformers transformers
import fitz
import random
import pandas as pd
import numpy as np
import torch
from time import perf_counter as timer

# from tqdm.auto import tqdm

def text_formatter(text : str) -> str:
  """Perfroms minor formatting on text """

  cleaned_text = text.replace("\n", " ").strip()

  return cleaned_text

def open_and_read_pdf(pdf_path : str) -> list[dict]:
  doc = fitz.open(pdf_path)
  pages_and_texts = []

  for page_number, page in (enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text=text)

    pages_and_texts.append({"page_number": page_number - 41,
                            "page_char_count" : len(text),
                            "page_word_count" : len(text.split(" ")),
                            "page_sentence_count_raw" : len(text.split(".")),
                            "page_token_count " : len(text)/4,
                            "text" : text})

  return pages_and_texts


# Here is an example pdf
pdf_path="/content/Resume_HammadAhmed (1).pdf"
pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)

# pages_and_texts[:2]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import random

random.sample(pages_and_texts, k=3)

[{'page_number': 284,
  'page_char_count': 979,
  'page_word_count': 167,
  'page_sentence_count_raw': 1,
  'page_token_count ': 244.75,
  'text': '4SAP ?JMQC\x01 l\x01 4NJCLB?\x01 \x12\x01 4NJCLB?\x01 N?AICR\x01 A ML R?GLQ\x01 \x14\x0f\x14\x12\x01 A ?JMPGCQ\x01 \x1e\x01\x12E\x01 ’GPQR\x01BGQA M TCP CB\x01GL\x01 \x12\x1a \x18\x17\x0f\x01"NNP M TCB\x01D MP\x01 SQC\x01GL\x01\x12\x1a\x1a\x19\x01GL\x01RFC\x01 6LG RC B\x014R? RCQ\x01?LB\x01 GL\x01\x12\x1a\x1a\x12\x01GL\x01 $?L?B?\x0f%CPGT CB\x01 DP MK\x01QSAP MQC\x01GL\x01 UFGAF\x01RFP CC\x01M D\x01G RQ\x01 FW BP M VW J\x01\t0 ) \x01 EP MSNQ\x01?P C\x01 PCNJ?A CB\x01@ W\x01 AFJMPGLC\x01\t $Jn \x0f\x01 4GV\x01FSLBP CB\x01RGKCQ\x01 QU CCRCP\x01RF?L\x01 QSE?P \x0f\x01 "%*\x1b\x01\x16\x01KE\x10IE\x01 @MB W\x01U CGE FR\x0f\x0b$?L\x01 @C\x01SQC B\x01GL\x01 A MMIGLE\x0f\x01 4RC TGMQGBC\x01 l\x01 4RC TG?\x01 l\x01 4UCCR\x01-C?D\x01 / \x10"\x01 %CPGT CB\x01DP MK\x01 QRC TG?\x01NJ?L R\x01D MSLB\x01 GL\x014MSRF\x01"KCPGA ?\x0f\x01 4RC TG?\x01P C@? SBG

In [ ]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)

df.head()

## Why would we care about token count ?

Token count is important to think about because:

1. Embedding models dont deal with infinite tokens.
2. LLMs dont deal with infinite tokens.

For example an embedding model may have been trained to embed sequences of 384 tokens into numerical space (`all-mpnet-base-v2`)

As for LLMs, they can not accept infinite tokens in their context window

### Further text processing (splitting pages into sentences)

Two ways to do this:
1. We've done this by splitting on "."
2. We can do this with a NLP library spaCy or NTLK

In [ ]:
from spacy.lang.en import English

In [ ]:
nlp = English()

# Add a sentencizer pipeline
nlp.add_pipe("sentencizer")


## Break the extracted text from each page into sentences

In [ ]:
from tqdm.auto import tqdm

for item in tqdm(pages_and_texts):
  item["sentences"] = list(nlp(item["text"]).sents)

  # Make sure all sentences are string ()
  item["sentences"] = [str(sentence) for sentence in item["sentences"]]

  item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/1 [00:00<?, ?it/s]

### Chunking our sentences together

The concept of splititng larger pieces of texts into smaller ones is often referred to as text splitting or chunking.

There is no 100% correct way to do this.

We'll keep it simple and split into groups of 10 sentences

There are frameworks such as LangChain which can help with this

In [ ]:
#Define split size to turn groups of sentences into chunks

num_sentence_chunk_size = 10

# Create a function to split lists of texts recursively into chunk size
# e.g [20] -> [10, 10]

def split_list(input_list : list[str],
               slice_size : int=num_sentence_chunk_size) -> list[list[str]]:

    return [input_list[i: i + slice_size] for i in range(0, len(input_list), slice_size)]


In [ ]:
test_list = list(range(25))
test_list

split_list(test_list)

## Break the sentences list into chunks

In [ ]:
# Loop through pages and texts and split sentences into chunks

for item in tqdm(pages_and_texts):
  item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                      slice_size=num_sentence_chunk_size)

  item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=2)

In [ ]:
df = pd.DataFrame(pages_and_texts)

df.describe().round(2)

### Splitting each chunk into its own item

We'd like to embed each chunk of sentences into its own numerical representation



In [ ]:
import re

# Split each chunk into its own item

pages_and_chunks = []

for item in tqdm(pages_and_texts):
  for sentence_chunk in item["sentence_chunks"]:
    chunk_dict = {}
    chunk_dict["page_number"] = item["page_number"]

    # Join the sentences together into a paragraph-like structure
    joined_sentence_chunk = "".join(sentence_chunk).replace(" ", " ").strip()

    joined_sented_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk)

    chunk_dict["sentence_chunk"] = joined_sentence_chunk

    #Get some stats on our chunks
    chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
    chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
    chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4

    pages_and_chunks.append(chunk_dict)

len(pages_and_chunks)

  0%|          | 0/1 [00:00<?, ?it/s]

2

In [ ]:
random.sample(pages_and_chunks, k=1)

[{'page_number': 67,
  'sentence_chunk': 'RMK W \r\x01?LB\x011FWQGMJME W \x01\x01Y \x01\x01\x17 \x18',
  'chunk_char_count': 33,
  'chunk_word_count': 7,
  'chunk_token_count': 8.25}]

## Filter chunks of text for short chunks

These chunks may not contain much useful information



In [ ]:
df = pd.DataFrame(pages_and_chunks)

In [ ]:
df

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count
0,-41,"Hafiz Hammad Ahmed Islamabad, Pakistan • hafi...",2331,376,582.75
1,-41,September 2023 - Present Officer (Operations)...,477,87,119.25


In [ ]:
# # Show random chunks with under 30 tokens in length

min_token_length = 30

# for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
#   print(f'Chunk token count : {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

In [ ]:
# Filter our Dataframe for wos with under 30 tokens
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")

## Embedding our text chunks


In [ ]:


# # Create a list of sentences
# sentences = ["The sentence transformer library provides an easy way to craete embeddings",
#              "Sentences can be embedded one by one or in a list",
#              "I love horses!"]

# #Sentences are encoded by calling model.encode()
# embeddings = embedding_model.encode(sentences)
# embeddings_dict = dict(zip(sentences, embeddings))

# # See the embeddings
# for sentence, embeddings in embeddings_dict.items():
#   print(f"Sentence : {sentence}")
#   print(f"Embeddings : {embeddings}")
#   print("")

In [ ]:
%%time

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device="cpu")
embedding_model.to(device)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 6.25 s, sys: 2 s, total: 8.25 s
Wall time: 17 s


SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Getting all the sentence chunks from the dict and then putting them into a list and then passing the list to the embedding model

In [ ]:
%%time

text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

## Embed in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32,
                                               convert_to_tensor=True)

# text_chunk_embeddings

CPU times: user 25.2 ms, sys: 0 ns, total: 25.2 ms
Wall time: 24.4 ms


In [ ]:
len(text_chunks)

2

In [ ]:
text_chunk_embeddings.shape

torch.Size([175, 768])

## Save embeddings to file


In [ ]:
# # Save embeddings to file
# text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)

# embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
# text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [ ]:
# #Import saved file and view

# text_chunks_and_embeddings_df_load = pd.read_csv(embeddings_df_save_path)

# text_chunks_and_embeddings_df_load.head()

In [ ]:
# # When loading from CSV
# text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=","))

# # When stacking
# embeddings = np.stack(text_chunks_and_embeddings_df["embedding"].tolist(), axis=0)
# embeddings = torch.Tensor(embeddings)

# print(embeddings.shape)  # Should be (N, 768)

In [ ]:
# import random
# import torch
# import numpy as np

# device ="cuda" if torch.cuda.is_available() else "cpu"

# #import texts and embeddings
# text_chunks_and_embeddings_df.head()

# #convert embedding column back to numpy array (it got converted to string when it saved to csv)
# text_chunks_and_embeddings_df["embedding"] = text_chunks_and_embeddings_df["embedding"].apply(lambda x: np.fromstring(x))

In [ ]:
# text_chunks_and_embeddings_df["embedding"]

In [ ]:
# embeddings = np.stack(text_chunks_and_embeddings_df["embedding"].tolist(), axis=0)
# embeddings = torch.Tensor(embeddings)

# RAG Search and Answer

In [ ]:
from sentence_transformers import util


In [ ]:
import textwrap

def print_wrapped(text, wrap_length=80):
  wrapped_text = textwrap.fill(text, wrap_length)
  print(wrapped_text)

In [ ]:
def retrieve_relevant_resources(query: str,
                                embeddings: torch.tensor,
                                model : SentenceTransformer=embedding_model,
                                n_resources_to_return : int=5,
                                print_time: bool=True):

  """Embedds a query with model and returns top k scores and indices from embeddings"""

  # Embed the query
  query_embedding = model.encode(query, convert_to_tensor=True)

  # get dot product scores
  start_time=timer()

  dot_scores = util.dot_score(query_embedding, embeddings)[0]

  end_time = timer()

  if print_time:
    print(f"[INFO] Time taken to get scores on {len(embeddings)} embeddings: {end_time - start_time} seconds")

  if len(pages_and_chunks) <= n_resources_to_return:
    n_resources_to_return = 2


  scores, indices = torch.topk(dot_scores,
                               k=n_resources_to_return)

  return scores, indices



In [ ]:
retrieve_relevant_resources(query="top macronutrients", embeddings=text_chunk_embeddings)

[INFO] Time taken to get scores on 2 embeddings: 9.521700008008338e-05 seconds


(tensor([-0.0155], device='cuda:0'), tensor([0], device='cuda:0'))

In [ ]:
def print_top_results_and_scores(query: str,
                                 embeddings:torch.tensor,
                                 pages_and_chunks: list[dict] = pages_and_chunks,
                                 n_resources_to_return :int=5):
  scores, indices = retrieve_relevant_resources(query=query,
                                                embeddings=embeddings)

  for score, idx in zip(scores, indices):
    print(f"Score :{score: }")
    print("Text: ")
    print_wrapped(pages_and_chunks[idx]["sentence_chunk"])
    print(f"Page number: {pages_and_chunks[idx]['page_number']}")
    print("\n")


## Getting an LLM

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("alpindale/gemma-2b")

#Putting the model to gpu
model = model.to("cuda")


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# input_text = "What are the macronutrients and what role do they play in human body ?"

# print(f"Input text: \n{input_text}")

# # Create a prompt template
# dialogue_template = [
#     {"role": "user",
#      "content": input_text}
# ]

# # Apply the chat template
# prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
#                                        tokenize=False,
#                                        add_generation_prompt=True)

# print(f"\nPrompt : {prompt}")

## Doing the RAG

In [ ]:
def prompt_formatter(query : str,
                     context_items : list[dict]):

  context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

  base_prompt = """
  Based on the following context, answer the query concisely and accurately. Think through the problem step by step, but only return the final answer.

  Context:
  {context}

  Query:
  {query}

  Answer:
  """

  base_prompt = base_prompt.format(context=context,
                               query=query)


  return base_prompt


In [ ]:
scores, indices

len(pages_and_chunks)

(tensor([0.1604, 0.1598, 0.1500, 0.1423, 0.1397], device='cuda:0'),
 tensor([ 73, 113,   6,  77, 140], device='cuda:0'))

In [ ]:
query = "list the experience of the candidate in the form of bullet points? "

scores, indices = retrieve_relevant_resources(query=query,
                                              embeddings=text_chunk_embeddings)

context_items = [pages_and_chunks[i] for i in indices]

prompt = prompt_formatter(query=query,
                          context_items=context_items)

print(prompt)

[INFO] Time taken to get scores on 2 embeddings: 7.734299970252323e-05 seconds

  Based on the following context, answer the query concisely and accurately. Think through the problem step by step, but only return the final answer.

  Context:
  - September 2023 - Present  Officer (Operations) – FAST Artificial Intelligence Society  ●  Part of Operations Team during Hackathon organized by AIO. ●  Managing Operations during the events and workshops.   TECHNICAL SKILLS    ● NumPy ● Pandas ● Scikit-learn ● Matplotlib ● PyTorch ● OpenCV ● Machine Learning ● Natural Language Processing  ● Computer Vision ● Deep Learning ● Web Scraping ● Data Cleaning ● Whisper ● Data Visualization ● Microsoft Office  ● C++ ● Python ● SQL
- Hafiz Hammad Ahmed  Islamabad, Pakistan • hafizhammadahmedofficial@gmail.com • 03330970437 • LinkedIn: HafizHammadAhmed  EDUCATION    September 2021 - Present  National University of Computer and Emerging Sciences, Islamabad  Bachelor of Science in Artificial Intelligence 

In [ ]:
%%time

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate an output of tokens

outputs = model.generate(**input_ids,
                         temperature=0.4,
                         do_sample=True,
                         max_new_tokens=256)

# Turn the output tokens into text

output_text = tokenizer.decode(outputs[0])
final_answer = output_text.split(prompt)[-1].strip()


print(f"Query: {query}")
# print(f"RAG Answer: \n{output_text.replace(prompt, '')}")
print(f"RAG Answer: {final_answer}")

Query: list the experience of the candidate in the form of bullet points? 
RAG Answer: ●  I have 3 years of experience in the field of Artificial Intelligence and Machine Learning.
  ●  I have worked on many projects and gained knowledge and experience in various fields.
  ●  I have a strong understanding of the fundamentals of AI and Machine Learning and how they can be applied in various industries.
  ●  I have worked on various projects in the field of Computer Vision, Natural Language Processing, and Machine Learning and have gained experience in building and deploying models.
  ●  I have also worked on projects in the field of Data Science and have gained experience in data cleaning, data analysis, and data visualization.
  ●  I have a strong understanding of the Python programming language and have used it to build and deploy models.
  ●  I have a strong understanding of the fundamentals of AI and Machine Learning and how they can be applied in various industries.
  ●  I have a s

#Streamlit app

In [ ]:
%%writefile app.py

import streamlit as st
import fitz
import pandas as pd
import torch
import re
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForCausalLM
from spacy.lang.en import English
from time import perf_counter as timer

device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize session state
if 'pages_and_chunks' not in st.session_state:
    st.session_state.pages_and_chunks = []
if 'text_chunk_embeddings' not in st.session_state:
    st.session_state.text_chunk_embeddings = None
if 'embedding_model' not in st.session_state:
    st.session_state.embedding_model = None
if 'tokenizer' not in st.session_state:
    st.session_state.tokenizer = None
if 'model' not in st.session_state:
    st.session_state.model = None


# Helper functions
def text_formatter(text: str) -> str:
    return text.replace("\n", " ").strip()

def open_and_read_pdf(pdf_file) -> list[dict]:
    doc = fitz.open(stream=pdf_file.read(), filetype="pdf")
    pages_and_texts = []
    for page_number, page in enumerate(doc):
        text = page.get_text()
        text = text_formatter(text=text)
        pages_and_texts.append({
            "page_number": page_number + 1,
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count_raw": len(text.split(".")),
            "page_token_count": len(text) / 4,
            "text": text
        })
    return pages_and_texts

def split_list(input_list: list[str], slice_size: int = 10) -> list[list[str]]:
    return [input_list[i: i + slice_size] for i in range(0, len(input_list), slice_size)]

def retrieve_relevant_resources(query: str, embeddings: torch.tensor, model: SentenceTransformer, n_resources_to_return: int = 5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    dot_scores = util.dot_score(query_embedding, embeddings)[0]
    scores, indices = torch.topk(dot_scores, k=n_resources_to_return)
    return scores, indices

def prompt_formatter(query : str,
                     context_items : list[dict]):
  context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

  base_prompt = f"""
  Based on the following context, answer the query concisely and accurately. Think through the problem step by step, but only return the final answer.

  Context:
  {context}

  Query:
  {query}

  Answer:
  """

  base_prompt = base_prompt.format(context=context,
                               query=query)

  return base_prompt


# Main Streamlit app
st.title("PDF Chat App")

# File uploader
uploaded_file = st.file_uploader("Choose a PDF file", type="pdf")

if uploaded_file is not None:
    if not st.session_state.pages_and_chunks:
        with st.spinner("Processing PDF..."):
            # Process PDF
            pages_and_texts = open_and_read_pdf(uploaded_file)

            # NLP processing
            nlp = English()
            nlp.add_pipe("sentencizer")

            for item in pages_and_texts:
                item["sentences"] = [str(sentence) for sentence in nlp(item["text"]).sents]
                item["page_sentence_count_spacy"] = len(item["sentences"])
                item["sentence_chunks"] = split_list(input_list=item["sentences"])
                item["num_chunks"] = len(item["sentence_chunks"])

            # Create chunks
            pages_and_chunks = []
            for item in pages_and_texts:
                for sentence_chunk in item["sentence_chunks"]:
                    chunk_dict = {
                        "page_number": item["page_number"],
                        "sentence_chunk": "".join(sentence_chunk).replace(" ", " ").strip(),
                    }
                    chunk_dict["sentence_chunk"] = re.sub(r'\.([A-Z])', r'. \1', chunk_dict["sentence_chunk"])
                    chunk_dict["chunk_char_count"] = len(chunk_dict["sentence_chunk"])
                    chunk_dict["chunk_word_count"] = len([word for word in chunk_dict["sentence_chunk"].split(" ")])
                    chunk_dict["chunk_token_count"] = len(chunk_dict["sentence_chunk"]) / 4
                    pages_and_chunks.append(chunk_dict)

            # Filter chunks
            min_token_length = 30
            pages_and_chunks_over_min_token_len = [chunk for chunk in pages_and_chunks if chunk["chunk_token_count"] > min_token_length]

            st.session_state.pages_and_chunks = pages_and_chunks_over_min_token_len

    if st.session_state.text_chunk_embeddings is None:
        with st.spinner("Generating embeddings..."):
            # Generate embeddings
            device = "cuda" if torch.cuda.is_available() else "cpu"
            st.session_state.embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", device=device)
            text_chunks = [item["sentence_chunk"] for item in st.session_state.pages_and_chunks]
            st.session_state.text_chunk_embeddings = st.session_state.embedding_model.encode(text_chunks, batch_size=32, convert_to_tensor=True)

    if st.session_state.tokenizer is None or st.session_state.model is None:
        with st.spinner("Loading language model..."):
            # Load language model
            # st.session_state.tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
            # st.session_state.model = AutoModelForCausalLM.from_pretrained("alpindale/gemma-2b").to(device)

            st.session_state.tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM-360M")
            st.session_state.model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM-360M").to(device)

    # Chat interface
    st.subheader("Chat with your PDF")
    query = st.text_input("Ask a question about the PDF:")

    if query:
        with st.spinner("Generating answer..."):
            scores, indices = retrieve_relevant_resources(
                query=query,
                embeddings=st.session_state.text_chunk_embeddings,
                model=st.session_state.embedding_model
            )
            context_items = [st.session_state.pages_and_chunks[i] for i in indices]
            prompt = prompt_formatter(query=query, context_items=context_items)

            input_ids = st.session_state.tokenizer(prompt, return_tensors="pt").to(device)
            outputs = st.session_state.model.generate(**input_ids, temperature=0.4, do_sample=True, max_new_tokens=256)
            output_text = st.session_state.tokenizer.decode(outputs[0])

            answer = output_text.replace(prompt, '')
            st.write(f"Answer: {answer}")

else:
    st.info("Please upload a PDF file to start chatting.")

Overwriting app.py


In [ ]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
import streamlit as st
from pyngrok import ngrok

# This is necessary for Colab to use ngrok
!pip install pyngrok
!ngrok authtoken 2jpmgNooKQ5ODvN5DxSSezDAsSN_5xRYK5e2SoY3VoTkRDkCN  # Replace with your ngrok auth token

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!streamlit run app.py &>/dev/null&

In [ ]:
from pyngrok import ngrok

# Create a tunnel configuration
conf = ngrok.conf.PyngrokConfig(auth_token="2jpmgNooKQ5ODvN5DxSSezDAsSN_5xRYK5e2SoY3VoTkRDkCN")

# Create an HTTPTunnel
http_tunnel = ngrok.connect(8501, "http", pyngrok_config=conf)
print(http_tunnel.public_url)

https://575f-34-125-194-15.ngrok-free.app


In [ ]:
!sleep 3
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://90b7-34-125-194-15.ngrok-free.app


In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()